In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['df.csv']


In [3]:
import fnmatch
from textblob import TextBlob
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk import pos_tag,pos_tag_sents
import regex as re
import operator
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.cross_validation import train_test_split  
from sklearn import metrics
from sklearn import svm
from sklearn.grid_search import GridSearchCV
import pickle
from nltk.corpus import stopwords

/opt/conda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [4]:
result=pd.read_csv('../input/df.csv')
result.sample(10)

,HotelReviews,Labels
1221,the james hotel is located close to everything...,truth
628,i have been meaning to write a review of the o...,truth
1068,this hotel was worth every cent. you have not ...,deceptive
1489,"i have been 4 times in chicago, and i have sta...",truth
602,"first let me say, i try not to be too critical...",truth
357,i was highly disappointed with my choice to st...,deceptive
215,this hotel was full of drunks. the lobby was f...,deceptive
93,extremely disappointed in our stay. this hotel...,deceptive
641,"upon first entering the hotel, we were greeted...",truth
670,we stayed at this hotel with our two teenage d...,truth


In [5]:
import nltk


In [6]:
stopwords_nltk_en = set(stopwords.words('english'))

In [7]:
stop = stopwords.words('english')

result['review_without_stopwords'] = result['HotelReviews'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [8]:
result.head()

,HotelReviews,Labels,review_without_stopwords
0,we stayed at the schicago hilton for 4 days an...,deceptive,stayed schicago hilton 4 days 3 nights confere...
1,hotel is located 1/2 mile from the train stati...,deceptive,hotel located 1/2 mile train station quite hik...
2,i made my reservation at the hilton chicago be...,deceptive,made reservation hilton chicago believing goin...
3,"when most people think hilton, they think luxu...",deceptive,"people think hilton, think luxury. know did. w..."
4,my husband and i recently stayed stayed at the...,deceptive,husband recently stayed stayed hilton chicago ...


In [10]:
def pos(review_without_stopwords):
    return TextBlob(review_without_stopwords).tags

In [11]:
os = result.review_without_stopwords.apply(pos)
os1 = pd.DataFrame(os)

os1 dataframe's each row will consist of a list of words with their respective part of speech. 

In [12]:
os1.head()

,review_without_stopwords
0,"[(stayed, VBN), (schicago, JJ), (hilton, NN), ..."
1,"[(hotel, NN), (located, VBD), (1/2, CD), (mile..."
2,"[(made, VBN), (reservation, NN), (hilton, NN),..."
3,"[(people, NNS), (think, VBP), (hilton, NN), (t..."
4,"[(husband, NN), (recently, RB), (stayed, VBD),..."


In [14]:
os1['pos'] = os1['review_without_stopwords'].map(lambda x:" ".join(["/".join(x) for x in x ]) )
os1.head()

,review_without_stopwords,pos
0,"[(stayed, VBN), (schicago, JJ), (hilton, NN), ...",stayed/VBN schicago/JJ hilton/NN 4/CD days/NNS...
1,"[(hotel, NN), (located, VBD), (1/2, CD), (mile...",hotel/NN located/VBD 1/2/CD mile/NN train/NN s...
2,"[(made, VBN), (reservation, NN), (hilton, NN),...",made/VBN reservation/NN hilton/NN chicago/NN b...
3,"[(people, NNS), (think, VBP), (hilton, NN), (t...",people/NNS think/VBP hilton/NN think/VBP luxur...
4,"[(husband, NN), (recently, RB), (stayed, VBD),...",husband/NN recently/RB stayed/VBD stayed/JJ hi...


In [15]:
result = result = pd.merge(result, os1,right_index=True,left_index = True)
result.head()

,HotelReviews,Labels,review_without_stopwords_x,review_without_stopwords_y,pos
0,we stayed at the schicago hilton for 4 days an...,deceptive,stayed schicago hilton 4 days 3 nights confere...,"[(stayed, VBN), (schicago, JJ), (hilton, NN), ...",stayed/VBN schicago/JJ hilton/NN 4/CD days/NNS...
1,hotel is located 1/2 mile from the train stati...,deceptive,hotel located 1/2 mile train station quite hik...,"[(hotel, NN), (located, VBD), (1/2, CD), (mile...",hotel/NN located/VBD 1/2/CD mile/NN train/NN s...
2,i made my reservation at the hilton chicago be...,deceptive,made reservation hilton chicago believing goin...,"[(made, VBN), (reservation, NN), (hilton, NN),...",made/VBN reservation/NN hilton/NN chicago/NN b...
3,"when most people think hilton, they think luxu...",deceptive,"people think hilton, think luxury. know did. w...","[(people, NNS), (think, VBP), (hilton, NN), (t...",people/NNS think/VBP hilton/NN think/VBP luxur...
4,my husband and i recently stayed stayed at the...,deceptive,husband recently stayed stayed hilton chicago ...,"[(husband, NN), (recently, RB), (stayed, VBD),...",husband/NN recently/RB stayed/VBD stayed/JJ hi...


### Training Phase

In [16]:
review_train, review_test, label_train, label_test = train_test_split(result['pos'],result['Labels'], test_size=0.2,random_state=13)

### Vectorize the Training and Testing data using TfidfVectorizer

In [17]:
tf_vect = TfidfVectorizer(lowercase = True, use_idf=True, smooth_idf=True, sublinear_tf=False)

X_train_tf = tf_vect.fit_transform(review_train)
X_test_tf = tf_vect.transform(review_test)

### Implementing the Model

#### SVM

In [18]:
def svc_param_selection(X, y, nfolds):
    Cs = [0.001, 0.01, 0.1, 1, 10]
    gammas = [0.001, 0.01, 0.1, 1]
    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(svm.SVC(kernel='linear'), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    return grid_search.best_params_

In [19]:
svc_param_selection(X_train_tf,label_train,5)

{'C': 1, 'gamma': 0.001}

In [20]:
clf = svm.SVC(C=10,gamma=0.001,kernel='linear')
clf.fit(X_train_tf,label_train)
pred = clf.predict(X_test_tf)

#### Predict on the Test Data

In [22]:
X_test_tf = tf_vect.transform(review_test)
pred = clf.predict(X_test_tf)

In [23]:
print(metrics.accuracy_score(label_test, pred))

0.884375


In [24]:
print (confusion_matrix(label_test, pred))

[[139  16]
 [ 21 144]]


In [25]:
print (classification_report(label_test, pred))

             precision    recall  f1-score   support

  deceptive       0.87      0.90      0.88       155
      truth       0.90      0.87      0.89       165

avg / total       0.88      0.88      0.88       320



### Test the Model using different Random States

- Random State 1

In [27]:
review_train, review_test, label_train, label_test = train_test_split(result['pos'],result['Labels'], test_size=0.2,random_state=1)

In [28]:
X_test_tf = tf_vect.transform(review_test)
pred = clf.predict(X_test_tf)

In [29]:
print(metrics.accuracy_score(label_test, pred))

0.984375


In [30]:
print (confusion_matrix(label_test, pred))

[[149   2]
 [  3 166]]


In [31]:
print (classification_report(label_test, pred))

             precision    recall  f1-score   support

  deceptive       0.98      0.99      0.98       151
      truth       0.99      0.98      0.99       169

avg / total       0.98      0.98      0.98       320



- Random State 10

In [33]:
review_train, review_test, label_train, label_test = train_test_split(result['pos'],result['Labels'], test_size=0.2,random_state=10)

In [34]:
X_test_tf = tf_vect.transform(review_test)
pred = clf.predict(X_test_tf)

In [35]:
print(metrics.accuracy_score(label_test, pred))

0.971875


In [36]:
print (confusion_matrix(label_test, pred))

[[157   3]
 [  6 154]]


In [37]:
print (classification_report(label_test, pred))

             precision    recall  f1-score   support

  deceptive       0.96      0.98      0.97       160
      truth       0.98      0.96      0.97       160

avg / total       0.97      0.97      0.97       320



- Random State 42

In [38]:
review_train, review_test, label_train, label_test = train_test_split(result['pos'],result['Labels'], test_size=0.2,random_state=42)

In [39]:
X_test_tf = tf_vect.transform(review_test)
pred = clf.predict(X_test_tf)

In [40]:
print(metrics.accuracy_score(label_test, pred))

0.975


In [41]:
print (confusion_matrix(label_test, pred))

[[166   2]
 [  6 146]]


In [42]:
print (classification_report(label_test, pred))

             precision    recall  f1-score   support

  deceptive       0.97      0.99      0.98       168
      truth       0.99      0.96      0.97       152

avg / total       0.98      0.97      0.97       320



#### Test the model with two reviews from Yelp


In [43]:
def test_string(s):
    X_test_tf = tf_vect.transform([s])
    y_predict = clf.predict(X_test_tf)
    return y_predict

In [44]:
test_string("The hotel was good.The room had a 27-inch Samsung led tv, a microwave.The room had a double bed")

array(['truth'], dtype=object)

In [48]:
test_string("My husband and I satayed for two nights at the Hilton Chicago,and enjoyed every minute of it! The bedrooms are immaculate,and the linnens are very soft. We also appreciated the free wifi,as we could stay in touch with friends while staying in Chicago. The bathroom was quite spacious,and I loved the smell of the shampoo they provided-not like most hotel shampoos. Their service was amazing,and we absolutely loved the beautiful indoor pool. I would recommend staying here to anyone.")

array(['deceptive'], dtype=object)

Well, the model predicted the above two reviews correctly. The first review is a Truthful review while the second review is a deceptive one!